In [2]:
import os
import sys
sys.path.append(r'C:\Study\SpeechAcquisitionModel')
print(os.getcwd())
import numpy as np
from scipy.spatial.distance import euclidean, minkowski
from sklearn.metrics import precision_recall_curve, average_precision_score
import yaml
from pprint import pprint

import random
import scipy
import torch

import dtw
import dtwalign
from fastdtw import fastdtw

from sklearn.manifold import TSNE
import pandas as pd

from src.speech_classification.audio_processing import SpeechCommandsDataCollector, AudioPreprocessorMFCCDeltaDelta
from src.siamese_net_sound_similarity.slstm_train import SiameseSpeechCommandsDataCollector
from src.siamese_net_sound_similarity.train_v2_with_settle_structure import SiameseDeepLSTMNet
from src.siamese_net_sound_similarity.train import SiameseLSTMNet

from src.siamese_net_sound_similarity.soft_dtw import SoftDTW


import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set()
sns.set(font_scale=2.0, rc={'text.usetex' : False})
sns.set_style("whitegrid")

# %matplotlib inline

C:\Study\SpeechAcquisitionModel\src\dtw


In [3]:
DISTANCE='euclidean'
STEP_PATTERN = 'typeIc'
CLOSED_END_STEP_PATTERN = "typeIc"
# DISTANCE=lambda x, y : minkowski(x,y, 1)

## Load data

In [4]:
config_path = r'C:\Study\SpeechAcquisitionModel\reports\seamise_net_01_09_2020_05_52_PM\config.yaml'
with open(config_path, 'r') as data_file:
    config = yaml.safe_load(data_file)
pprint(config)

{'batch_norm': True,
 'dct_coefficient_count': 13,
 'dist': 'l1',
 'dropout': 0.3,
 'hidden_fc': [1024, 1024, 1024],
 'hidden_reccurent': [512, 512, 512],
 'label_count': 32,
 'learning_rate': 0.0005,
 'loss_type': 'sdtw',
 'margin': 0.4,
 'mfcc_num': 39,
 'mini_batch_size': 64,
 'open_end': False,
 'save_dir': "r'C:\\Study\\SpeechAcquisitionModel\\models\\siamese_net_sound_similarity'",
 'train_steps': 100000,
 'triplet_anneal_b': 1500,
 'triplet_anneal_k': 0.0025,
 'wanted_words': ['bed',
                  'bird',
                  'cat',
                  'dog',
                  'down',
                  'eight',
                  'five',
                  'four',
                  'go',
                  'happy',
                  'house',
                  'left',
                  'marvin',
                  'nine',
                  'no',
                  'off',
                  'on',
                  'one',
                  'right',
                  'seven',
             

In [5]:
wanted_words = config['wanted_words']

wanted_words_combined = wanted_words

model_settings = config
query_length = 100

In [6]:
preproc = AudioPreprocessorMFCCDeltaDelta(numcep=model_settings['dct_coefficient_count'], winlen=model_settings['winlen'],
                                     winstep=model_settings['winstep'])

data_iter = SiameseSpeechCommandsDataCollector(preproc,
                                        data_dir=r'C:\Study\Speech_command_classification\speech_dataset',
                                        wanted_words=wanted_words_combined,
                                        testing_percentage=10,
                                        validation_percentage=10
                                        )

index_to_word = {v:k for k,v in data_iter.word_to_index.items()}

# Gather Data for analysis

In [7]:
n_mini_batch_size = 1000

# gather data


data = data_iter.get_data(n_mini_batch_size, 0, 'validation')
labels = data['y']

data1 = data_iter.get_data(n_mini_batch_size, 0, 'validation')
labels1 = data1['y']
idx = np.random.randint(0, len(labels1), len(labels1))
data1_x = data1['x'][idx]
data1['x'] = data1_x
labels1 =  np.array(data1['y'])[idx]
data1['y'] = labels1

# labesl1 = data1['y']

# duplicates = data_iter.get_duplicates(labels, 0, 'testing')
# assert np.any(labels == duplicates['y'])
# duplicates['x'] = duplicates['x'][:n_duplicates_size]
# print(duplicates['x'].shape)

# non_duplicates = data_iter.get_nonduplicates(labels, 0, 'testing')
# assert np.any(labels != non_duplicates['y'])

# y_true = np.concatenate((np.ones((n_duplicates_size)), np.zeros((n_mini_batch_size))))
y_true = labels==labels1
print(y_true)
print(labels)
print(labels1)
print(sum(y_true))
print(data['x'].shape)
dists = []

[False False False False False False False False False False False False
 False False False False False  True False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True  True False False
 False False False False False False False False False False False False
 False False  True False False False False False False False False False
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True False False False False False False
  True False False False False False False False False  True False False
 False False False False False False False False Fa

 30. 10. 19.  5. 22. 27. 30.  7. 26. 11.]
[29. 18. 21.  8.  8.  3. 13. 11. 12. 20. 23. 27. 10. 18.  3.  9. 21. 10.
  2. 18. 23. 30. 17.  5. 21. 15.  3.  6.  3.  5. 25. 14. 13.  9. 30. 21.
 24. 24.  7. 28. 26. 18. 29. 30. 27. 31. 13.  4.  5. 27. 28. 13. 31. 18.
 18. 16. 24. 23. 17. 31. 18. 19. 13. 17. 10. 21. 15. 15. 18.  8.  5. 19.
 25. 16.  6. 17. 11.  2. 17. 25. 16. 20. 20. 20. 11. 27. 25. 19.  7. 18.
 18. 25. 14.  3.  2. 12.  5. 26.  2.  9. 24. 28. 23. 27. 16. 13. 23. 18.
  4. 24.  3. 17. 14. 13. 27. 20.  5. 10. 17. 21.  9.  7. 20.  3. 26. 25.
 31.  7. 27. 31. 23. 21. 19.  8. 26. 29. 12. 25. 21.  4. 16. 11. 23. 11.
 11. 30. 31. 30. 25. 15. 31.  5. 31. 31. 13. 16.  8.  2. 30. 11. 25.  9.
 16. 24.  3. 12. 20. 17. 24. 17. 31. 10. 10. 18.  7. 20.  7. 12. 20. 13.
 28.  7. 16.  9.  2. 15.  7. 13.  5. 16.  5. 21. 17. 25. 15. 22. 19.  5.
 28.  9. 10. 28. 15. 11. 23. 11. 23. 12. 12. 26. 22. 18. 28. 17. 30. 22.
 31. 30. 21. 16. 27.  7. 23. 16. 28. 23. 29. 22. 16. 18. 12. 20. 17. 16.
 26. 30. 

## 1. MFCC Precision/Recall

In [8]:

# initialize dist_lists

dtw = []

for i in range(n_mini_batch_size):
    print(i, end='\r')
    x = data['x'][i].squeeze()
    y = data1['x'][i].squeeze()
    dtw.append(dtwalign.dtw(x, y, dist=DISTANCE, step_pattern=STEP_PATTERN, open_end=False, dist_only=True).normalized_distance)

mfcc_dtw_dist = np.array(dtw)
dists.append(mfcc_dtw_dist)

# 2. Improving DTW using latent variables from LSTM classifier as a signal

In [9]:
# load nn

# dists = []
# dists.append(mfcc_dtw_dist)

# nn_ce = r'C:\Study\SpeechAcquisitionModel\reports\seamise_net_01_20_2020_12_30_PM\net_7500_0.9765625.net'
nn_ce = r'C:\Study\SpeechAcquisitionModel\reports\seamise_net_01_18_2020_03_58_PM\net_4500_0.8984375.net'

nn_ce_l2 = r'C:\Study\SpeechAcquisitionModel\reports\seamise_net_01_18_2020_03_58_PM\net_5500_0.8984375.net'
# nn_ce_dtw = r'C:\Study\SpeechAcquisitionModel\reports\seamise_net_01_19_2020_01_42_PM\net_29500_0.80078125.net'
nn_ce_dtw = r'C:\Study\SpeechAcquisitionModel\reports\seamise_net_01_18_2020_05_38_PM\net_48000_0.96875.net'

nn_ce_cos = r'C:\Study\SpeechAcquisitionModel\reports\seamise_net_01_18_2020_05_38_PM\net_48500_0.9921875.net'

nn_dtw = r'C:\Study\SpeechAcquisitionModel\reports\seamise_net_01_17_2020_08_47_PM\net_44000_0.0546875.net'
# nn_dtw = r'C:\Study\SpeechAcquisitionModel\reports\seamise_net_01_22_2020_12_57_PM\net_49500_0.01953125.net' #temp

nn_cos_hinge = r'C:\Study\SpeechAcquisitionModel\reports\seamise_net_01_17_2020_04_00_PM\net_8000_0.046875.net'
nn_l2 = r'C:\Study\SpeechAcquisitionModel\reports\seamise_net_01_18_2020_01_26_PM\net_15500_0.046875.net'
# net_237_0.10546875

nn_fnames = [nn_cos_hinge, nn_dtw, nn_l2, nn_ce, nn_ce_l2, nn_ce_cos, nn_ce_dtw]
nn_names = ['Cos-triplet', 'Soft-DTW-triplet', 'L2-triplet', 'CE', 'CE+L2-triplet', 'CE+Cos-triplet', 'CE+Soft-DTW-triplet']


# nn_fnames = [nn_cos_hinge, nn_dtw]
# nn_names = ["COS_HINGE", 'DTW']

model_names = ['MFCC']
model_names.extend(nn_names)


### 2.2 - SOFT-DTW

In [10]:

soft_dtw_loss_close_end = SoftDTW(open_end=False, dist='l1')
dist_funcs = {
    'sdtw': soft_dtw_loss_close_end,
    'l2': lambda x, y: torch.sum((x - y)**2, dim=-1)[-1],
    'cos_hinge': lambda x, y: 1 - torch.nn.CosineSimilarity(dim=0)(x[-1, :], y[-1, :]),
    'ce': soft_dtw_loss_close_end
}


for i in range(len(nn_fnames)):
    nn_fname = nn_fnames[i]
    config_path = os.path.join(os.path.dirname(nn_fname), 'config.yaml')
    with open(config_path, 'r') as data_file:
        config = yaml.safe_load(data_file)
   
    loss_type = config['loss_type']
    dist_func = dist_funcs[loss_type]
    nn = torch.load(nn_fname).cpu()
    nn.eval()

    print(f"############################################################################\n\
# Model {model_names[i+1]}\n\
############################################################################")

    PREFIX_LENS = [100]

    for PREFIX_LEN in PREFIX_LENS:
        print(f"############################################################################\n\
# PREFIX LENGTH : {PREFIX_LEN}\n\
############################################################################")
        nn_input = torch.from_numpy(data['x'][:, :PREFIX_LEN, :]).float()


        z, *_ = nn.single_forward(nn_input)
        z = z

        z1, *_ = nn.single_forward(torch.from_numpy(data1['x']).float())
        z1 = z1


        ###########################################################################
        # DTW
        ###########################################################################

#         soft_dtw_loss_open_end = SoftDTW(open_end=True, dist='l1')
#         soft_dtw_loss_close_end = SoftDTW(open_end=False, dist='l1')

        ##########################################################################
        # OPEN END = False
        ##########################################################################

        # initialize dist_lists

        dtw_z = []

        for i in range(n_mini_batch_size):
            print(i, end='\r')
            x = z[i].squeeze()
            y = z1[i].squeeze()

            d = dist_func(x, y)
            dtw_z.append(d.detach().cpu().numpy())

        dtw_dist = np.array(dtw_z)
        dists.append(dtw_dist)
        
        del z, z1, _


C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:453: SourceChangeWarning: source code of class 'src.siamese_net_sound_similarity.train_v2_with_settle_structure.SiameseDeepLSTMNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


############################################################################
# Model Cos-triplet
############################################################################
############################################################################
# PREFIX LENGTH : 100
############################################################################
############################################################################
# Model Soft-DTW-triplet
############################################################################
############################################################################
# PREFIX LENGTH : 100
############################################################################


C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:453: SourceChangeWarning: source code of class 'src.siamese_net_sound_similarity.train_v2_with_settle_structure.SiameseDeepLSTMNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


############################################################################
# Model L2-triplet
############################################################################
############################################################################
# PREFIX LENGTH : 100
############################################################################
############################################################################
# Model CE
############################################################################
############################################################################
# PREFIX LENGTH : 100
############################################################################
############################################################################
# Model CE+L2-triplet
############################################################################
############################################################################
# PREFIX LENGTH : 100
###################################

# 3. mAP

## 3.1 Precision_Recall Curve

In [14]:
plt.figure(figsize=(6,6), dpi=1200)

# no skill model
precision, recall, _ = precision_recall_curve(y_true, np.zeros(len(y_true)), pos_label=1)
ax = sns.lineplot(recall, precision)
ax.lines[0].set_linestyle("--")


styles = ['dotted', 'solid', 'dashdot']
k = 0
for i in range(len(model_names)):
    precision, recall, thresholds = precision_recall_curve(y_true, -dists[i], pos_label=1)
    print(model_names[i])
    if model_names[i].startswith('CE') or model_names[i].startswith('MFCC'):
        continue
        st = '--'
    else:
        st = '-'
    print(st)
    ax = sns.lineplot(recall, precision, label=model_names[i], linewidth=3, ci =None )
    ax.lines[-1].set_linestyle(styles[k])
    k+=1
plt.legend(bbox_to_anchor=(1.05, 1), loc=2)
plt.show()

MFCC
Cos-triplet
-
Soft-DTW-triplet
-
L2-triplet
-
CE
CE+L2-triplet
CE+Cos-triplet
CE+Soft-DTW-triplet


In [16]:
plt.figure(figsize=(6,6), dpi=1200)

# no skill model
precision, recall, _ = precision_recall_curve(y_true, np.zeros(len(y_true)), pos_label=1)
ax = sns.lineplot(recall, precision)
ax.lines[0].set_linestyle("--")

styles = ['dashed', 'dotted', 'solid', 'dashdot']
k = 0
for i in range(len(model_names)):
    precision, recall, thresholds = precision_recall_curve(y_true, -dists[i], pos_label=1)
    print(model_names[i])
    if model_names[i].startswith('CE'):
        st = '-'
    else:
        continue
    print(st)
    ax = sns.lineplot(recall, precision, label=model_names[i], linewidth=3, ci =None )
    ax.lines[-1].set_linestyle(styles[k])
    k+=1
plt.legend(bbox_to_anchor=(1.05, 1), loc=2)
plt.show()

MFCC
Cos-triplet
Soft-DTW-triplet
L2-triplet
CE
-
CE+L2-triplet
-
CE+Cos-triplet
-
CE+Soft-DTW-triplet
-


## 3.2 Average Precision 

In [13]:
for i in range(len(model_names)):
    average_precision = average_precision_score(y_true, -dists[i])
    print(model_names[i], average_precision)
#     precision, recall, thresholds = precision_recall_curve(y_true, models_dists[i])

MFCC 0.04835008502233193
Cos-triplet 0.6824955597312906
Soft-DTW-triplet 0.6480041785330797
L2-triplet 0.4970692330910392
CE 0.7208947826676618
CE+L2-triplet 0.7281154867379794
CE+Cos-triplet 0.8972547480555466
CE+Soft-DTW-triplet 0.9097186386873051
